In [ ]:

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
import h5py
import warnings

warnings.filterwarnings('ignore')

def get_features(MODEL, width, height, lambda_func=None):
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)

    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    # 注意 train 和 test 是图片存储路径
    train_generator = gen.flow_from_directory("E:\dog_cat\train", (width, height), shuffle=False,
                                              batch_size=16)
    test_generator = gen.flow_from_directory("E:\dog_cat\test1", (width, height), shuffle=False,
                                             batch_size=16, class_mode=None)

    train = model.predict_generator(train_generator,train_generator.nb_sample)
    test = model.predict_generator(test_generator,test_generator.nb_sample)
    if MODEL==VGG16:
        with h5py.File('data_VGG16.h5','w' ) as h:
            h.create_dataset("train", data=train)
            h.create_dataset("test", data=test)
            h.create_dataset("label", data=train_generator.classes)
    elif MODEL==Xception:
        with h5py.File('data_Xception.h5','w' ) as h:
            h.create_dataset("train", data=train)
            h.create_dataset("test", data=test)
            h.create_dataset("label", data=train_generator.classes)
    else: 
         with h5py.File('data_InceptionV3.h5','w' ) as h:
            h.create_dataset("train", data=train)
            h.create_dataset("test", data=test)
            h.create_dataset("label", data=train_generator.classes)

get_features(VGG16, 224, 224)
get_features(InceptionV3, 299, 299, inception_v3.preprocess_input)
get_features(Xception, 299, 299, xception.preprocess_input)

Using TensorFlow backend.
c:\users\雪夜\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\雪夜\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\雪夜\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



In [ ]:
import h5py
import numpy as np
from sklearn.utils import shuffle
from keras.models import *
from keras.layers import *

np.random.seed(12345678)
X_train = []
X_test = []

for filename in ["data_VGG16.h5", "data_Xception.h5", "data_InceptionV3.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
X_train, y_train = shuffle(X_train, y_train)


In [ ]:

input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='softmax')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=128, nb_epoch=8, validation_split=0.2)
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    epochs = range(len(acc))
    plt.plot(epochs, acc, 'b')
    plt.plot(epochs, val_acc, 'r')
    plt.legend(["acc", "val_acc"], loc='best')
    plt.title('Training and validation accuracy')
    plt.show()
    loss = history.history['loss']
    val_loss = history.history['val_loss']   
    plt.plot(epochs, loss, 'b')
    plt.plot(epochs, val_loss, 'r')
    plt.legend(["loss", "val_loss"], loc='best')
    plt.title('Training and validation loss')
    plt.show()

plot_training(history)